In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(200000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

In this notebook i want to playaround with using multiple keys in a index. 
I hope to find that i could have a query with different columns used in a where clause and i could have a single index on of these and it should be faster than having none, but having two keys should improve the performance

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
execute_query("SELECT count(1) from products where price < 50", database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
execute_query("SELECT count(1) from products where size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_price ON products(size, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
execute_query("select count(*) from products where price < 50", database=get_database_name(), print_as_df=True, show_metrics=False)
execute_query("select count(*) from products where size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)

So this seems to show that using a single index can speed thing up (depending on the index and how you query) but two would still be better.
And just for fun i want to add another key to the index to see if this would make it slower. My assumption is that it shouldn't affect the query time

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_price_and_name ON products(size, price, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large' AND price < 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

It does actually seem to affect it quite a bit. And i dont understand why.

What looks weird is that the rows from the explain is twice s big eventhough the query is the same. 
I dont know if the extra column in the index will mess up the execution planners calulation.

I just want to quickly add that even though a key is indexed it matters how it is used in a search because it can be useless

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_price_and_name ON products(size, price, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price = 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

Here it uses a full table scan because the prices is sorted behind the size which means that if you dont do any filtering on the size the price part cannot be used on its own.